# MobileNetV2 Pruning & Quantization (No Library)

In [1]:
import sys
sys.path.append('.')

import torch
import torch.nn as nn
from collections import OrderedDict
from models.mobilenetv2 import MobileNetV2

# ==================== Configuration ====================
CKPT_PATH = './checkpoints/mobilenetv2_ckpt_best.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==================== Load Model ====================
model = MobileNetV2()
ckpt = torch.load(CKPT_PATH, map_location='cpu')

# Handle DDP state_dict
state_dict = ckpt['net']
if list(state_dict.keys())[0].startswith('module.'):
    state_dict = OrderedDict([(k.replace('module.', ''), v) for k, v in state_dict.items()])
model.load_state_dict(state_dict)
model = model.to(DEVICE)
model.eval()

# ==================== Model Info ====================
print('=' * 60)
print('MobileNetV2 for CIFAR-10 - Model Information')
print('=' * 60)
print(f"Checkpoint: {CKPT_PATH}")
print(f"Best Accuracy: {ckpt.get('acc', 'N/A'):.2f}%" if 'acc' in ckpt else f"Accuracy: N/A")
print(f"Trained Epochs: {ckpt.get('epoch', 'N/A')}")
print(f"Device: {DEVICE}")
print()

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model Size: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print()

# Layer-wise breakdown
print('Layer-wise Parameter Count:')
print('-' * 60)
conv_layers = []
depthwise_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        params = sum(p.numel() for p in module.parameters())
        is_depthwise = module.groups == module.in_channels and module.groups > 1
        layer_type = 'DWConv' if is_depthwise else 'Conv2d'
        if is_depthwise:
            depthwise_layers.append((name, module, params))
        else:
            conv_layers.append((name, module, params))
        print(f"{name:40s} | {layer_type:6s} | in={module.in_channels:4d}, out={module.out_channels:4d}, k={module.kernel_size[0]}, g={module.groups:3d} | {params:,} params")
    elif isinstance(module, nn.Linear):
        params = sum(p.numel() for p in module.parameters())
        print(f"{name:40s} | Linear | in={module.in_features:4d}, out={module.out_features:4d}              | {params:,} params")

print()
print('=' * 60)
print(f"Total Conv2d Layers (non-depthwise): {len(conv_layers)}")
print(f"Total Depthwise Conv Layers: {len(depthwise_layers)}")
print(f"Total Conv2d Parameters: {sum(p for _, _, p in conv_layers):,}")
print(f"Total Depthwise Parameters: {sum(p for _, _, p in depthwise_layers):,}")
print('=' * 60)

MobileNetV2 for CIFAR-10 - Model Information
Checkpoint: ./checkpoints/mobilenetv2_ckpt_best.pth
Best Accuracy: 93.85%
Trained Epochs: 99
Device: cuda

Total Parameters: 2,296,922
Trainable Parameters: 2,296,922
Model Size: 8.76 MB (FP32)

Layer-wise Parameter Count:
------------------------------------------------------------
conv1                                    | Conv2d | in=   3, out=  32, k=3, g=  1 | 864 params
layers.0.conv1                           | Conv2d | in=  32, out=  32, k=1, g=  1 | 1,024 params
layers.0.conv2                           | DWConv | in=  32, out=  32, k=3, g= 32 | 288 params
layers.0.conv3                           | Conv2d | in=  32, out=  16, k=1, g=  1 | 512 params
layers.0.shortcut.0                      | Conv2d | in=  32, out=  16, k=1, g=  1 | 512 params
layers.1.conv1                           | Conv2d | in=  16, out=  96, k=1, g=  1 | 1,536 params
layers.1.conv2                           | DWConv | in=  96, out=  96, k=3, g= 96 | 864 params
la